In [7]:
import shapely.geometry
import pytest, time, requests

from IPython.display import display
from ipywidgets import Layout, Button, Text, DatePicker
from ipyleaflet import *
from osprey_flask_app import create_app


In [8]:
def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        lat = round(kwargs.get("coordinates")[0], 5)
        lon = round(kwargs.get("coordinates")[1], 5)

        if in_polygon(lat, lon):
            clear_previous_marker()

            latitude.value = f"{lat}"
            longitude.value = f"{lon}"

            m.add_layer(Marker(location=kwargs.get("coordinates"), name="Marker"))


def handle_button(arg):
    print("The button has been clicked")

In [9]:
def clear_previous_marker():
    for i in range(1, len(m.layers)):
        if m.layers[i].name == "Marker":
            m.remove_layer(m.layers[i])


def in_polygon(lat, lon):

    point = shapely.geometry.Point(lat, lon)

    peace_polygon = shapely.geometry.polygon.Polygon(p.locations)
    fraser_polygon = shapely.geometry.polygon.Polygon(f.locations)
    columbia_polygon = shapely.geometry.polygon.Polygon(c.locations)

    if (
        peace_polygon.contains(point)
        or fraser_polygon.contains(point)
        or columbia_polygon.contains(point)
    ):
        return True

    return False

def date_widget(descr):
    return DatePicker(descriptio=descr, disabled=False)

In [10]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = "Default"

satellite = basemap_to_tiles(basemaps.Gaode.Satellite)
satellite.base = True
satellite.name = "Satellite"

m = Map(
    basemap=mapnik,
    center=(52, -120),
    zoom=5,
    layout=Layout(width="50%", height="800px", crs="EPSG3857"),
    layers=[satellite, mapnik],
)

p = Polygon(
    locations=[
        (53.0312, -127.8437),
        (58.1562, -127.8437),
        (58.1562, -115.9062),
        (53.0312, -115.9062),
    ],
    color="blue",
    name="Peace",
)
f = Polygon(
    locations=[
        (48.5312, -128.4687),
        (56.4687, -128.4687),
        (56.4687, -117.5312),
        (48.5312, -117.5312),
    ],
    color="red",
    name="Fraser",
)
c = Polygon(
    locations=[
        (41.0312, -124.9687),
        (53.1562, -124.9687),
        (53.1562, -109.7187),
        (41.0312, -109.7187),
    ],
    color="green",
    name="Columbia",
)

legend = LegendControl(
    {"Peace": "blue", "Fraser": "red", "Columbia": "green"},
    name="Watersheds",
    position="topright",
)

latitude = Text(
    placeholder="Enter the Latitude", description="Latitude:", disabled=False
)

longitude = Text(
    placeholder="Enter the Longitude", description="Longitude:", disabled=False
)

layer_control = LayersControl()

button = Button(
    description="Run",
    button_style="success",
    disabled=False,
    tooltip="Click 'Run' to start the Osprey Flask App.",
)
button.on_click(handle_button)

start_date = date_widget("Start Date")
end_date = date_widget("End Date")

In [11]:
m.add_layer(p)
m.add_layer(f)
m.add_layer(c)

m += layer_control

m.on_interaction(handle_click)
m.add_control(legend)

display(m)
display(start_date, end_date)
display(latitude, longitude)
display(button)

Map(center=[52, -120], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

DatePicker(value=None)

DatePicker(value=None)

Text(value='', description='Latitude:', placeholder='Enter the Latitude')

Text(value='', description='Longitude:', placeholder='Enter the Longitude')

Button(button_style='success', description='Run', style=ButtonStyle(), tooltip="Click 'Run' to start the Ospre…

The button has been clicked


In [12]:
def client():
    flask_app = create_app()

    # Create a test client using the Flask application configured for testing
    with flask_app.test_client() as testing_client:
        # Establish an application context
        with flask_app.app_context():
            yield testing_client
            
def full_rvic_test(url, client, valid_input=True):
    #input_params = urlencode(url)
    input_url = f"/osprey/input?{url}"
    
    print(input_url)

    input_response = client.get(url)
    if valid_input:
        assert input_response.status_code == 202
    else:
        assert input_response.status_code == 400
        return

    status_url = input_response.data.split()[-1].decode("utf-8")
    status_response = client.get(status_url)

    timeout = 1800  # Time to timeout in seconds
    for i in range(timeout):
        if status_response.data != b"Process is still running.":  # Process is completed
            break
        time.sleep(1)
        status_response = client.get(status_url)
    assert b"Process completed." in status_response.data

    output_url = status_response.data.split()[-1].decode("utf-8")
    output_response = client.get(output_url)
    streamflow_path = output_response.headers.get("Location")
    assert requests.get(streamflow_path).status_code == 200

In [7]:
def build_url(start, end, lat, lon):
    
    start = "&run_startdate=" + str(start)
    end = "&stop_date=" + str(end)
    lon = "&lons=" + lon
    lat = "&lats=" + lat
    url = "case_id=sample"+ start + end + lon + lat
    
    print(url)
    
    return url
    
    
    
url = build_url(start_date.value, end_date.value, latitude.value, longitude.value)
full_rvic_test(url, client, valid_input=True)

case_id=sample&run_startdate=2022-07-05&stop_date=2022-07-07&lons=-122.12666&lats=51.67619
/osprey/input?case_id=sample&run_startdate=2022-07-05&stop_date=2022-07-07&lons=-122.12666&lats=51.67619


AttributeError: 'function' object has no attribute 'get'